# Data Preprocessing Tools

## Importing the libraries



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Importing the dataset

In [3]:
daturl = 'https://raw.githubusercontent.com/sakuronohana/my_datascience/master/udemy/mlaz/Part%201%20-%20Data%20Preprocessing/Python/Data.csv'

dataset = pd.read_csv(daturl)

### Selecting data

Wenn wir Supervised Learning durchführen, dann besteht der Datensatz meist aus einem **Label** (y, abhängigen Variable / dependent Variable) und den **Merkmalen** (Features, X, unabhängigen Variablen / independent Variables).
Es ist Standard, das in einem Datensatz das Label meist in der hintersten Spalte ist.

Wir importieren nun die die Labels und Features in verschieden Matrixen

In [11]:

X = dataset.iloc[:,:-1].values # Erste Doppelpunkt ist der Range mit Rows und zweiter Doppelpunkt der Range mit Columns. Mit Values erstellen wir eine Matrix mit den Werten ohne Header
y = dataset.iloc[:,-1].values

In [16]:
print('Hier sind die Labels:',y)
print('Hier sind die Features:\n',X)

Hier sind die Labels: ['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']
Hier sind die Features:
 [['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Taking care of missing data

Es kommt des öfteren vor, dass in einem Datensatz Werte fehlen. Die Art und Weise wie man mit fehlenden Daten umgeht ist unterschiedlich und geht von den Werten ersetzen (mit Mean, Median usw.) bis löschen, was bei einem grossen Datensatz meist kein Problem darstellt jedoch bei einem kleinen schon.

Im folgenden Beispiel werden wir die fehlenden Werte mittels arithm. Mittelwert ersetzen. Dazu verwenden wir eines der beliebtesten ML Tools **scikit-learn**. 

In [19]:
# Schauen wir mal zuerst wo wir fehlende Wert haben
dataset.isna()

,Country,Age,Salary,Purchased
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,True,False
5,False,False,False,False
6,False,True,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [23]:
# Um alle fehlenden Werte in den Features zu ersetzen verwenden wir SimpleImputer
from sklearn.impute import SimpleImputer

# Zuerst ersellen eine Instanz der Klasse SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # Der erste Wert in der Klammer bedeutet, was wir ersetzen und der zweite Wert mit was

# Nun benötigen wir noch eine Funktion mit welcher wir die Klasse auf unseren Datensatz anwenden können. Hierzu steht die Funktion fit
imputer.fit(X[:,1:3]) #im 



SyntaxError: ignored

## Encoding categorical data

### Encoding the Independent Variable

### Encoding the Dependent Variable

## Splitting the dataset into the Training set and Test set

## Feature Scaling